# Sentence Completion
This notebook is used for a final project as a part of the CS 4980 Natural Language Processing course at the Milwaukee School of Engineering during the spring term of 2023. This notebook was created by [Grant Fass](grantfass@gmail.com) and [Nicholas Kaja](kajan@msoe.edu). The following notebook will explore the problem of natural language sentence completion.

Natural language sentence completion involves determining what word best fits in a blank present in a sentence. This type of question is typically found on the Scholastic Aptitude Test (SAT). It is useful because it can measure the performance of a language model (LM) on questions that educational experts deem important. LMs that perform well on this type of problem will likely perform better on broader tasks.

The primary method for forming a sentence completion model is to compute the probability of each possible sentence tehn choose the most probable option. This probability computation can be done using n-grams, Latent Semantic Analysis (LSA), and Syntactic Dependency Trees. Some research has also been done into combining these with methods of preserving long-range dependencies in the sentences. Of these options, N-grams is one of the easiest starting points due to its ease of implementation and understanding. N-grams also allow for sufficient variations over the base model such as different N-gram algorithms, different values of N, and different methods of tokenization.

We will be constructing our models from a dataset of Khan Academy lecture transcripts. This dataset was scraped using BeautifulSoup during January of 2023 by Nicholas Kaja as a part of a senior design project. The performance of our model will be evaluated on a [SAT Question Dataset](https://github.com/ctr4si/sentence-completion/tree/master/data/completion). After evaluation, we plan to take our best performing model and apply it to sentence generation. This will allow us to get a better feel for how well it performs. Once our model is working, we also plan to implement some additional features such as Named Entity Recognition (NER). If there is enough time we also plan to investigate using the [OpenAI tokenizer](https://platform.openai.com/tokenizer) instead of the [NLTK Word Tokenizer](https://www.nltk.org/api/nltk.tokenize.html) or the [SpaCy Tokenizer](https://spacy.io/api/tokenizer). The OpenAI tokenizer has a [python package found on github](https://github.com/openai/tiktoken).

For more information please see the Data Collection And Processing document as well as the Project Background document. Both of these documents can be found in the repository under the Sentence Completion directory.

## Imports
Below are sample imports based on other NLP tasks.

In [ ]:
from pathlib import Path
from collections import Counter, defaultdict
import pandas as pd
import random
from tqdm import tqdm
import nltk
import re
import contractions
import unidecode
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import f1_score, recall_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
import math
import numpy as np
import copy

nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt",
])

lemmatizer = WordNetLemmatizer()
sia = SentimentIntensityAnalyzer()
encoder = LabelEncoder()